# A : Setup

##A.1 Authentication and Checking User Details.

In [2]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [3]:
# Getting the email account that was used to login to this session
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
print(gcloud_tokeninfo['email'])

cfemilioc@gmail.com


##A.2 Setup the Project Parameters.

In [5]:
# Change the following if you are not in the rsa-dev-10 project
# These parameters are generated outside of this notebook
# Project: rsa-mla-prod

# Project: rsa-dev-10
project_id = 'rsa-mono-site5'
bucket_id = 'rsa-mono-site5'
# ip_addr_name = 'rsa-static-west'
# project_id = 'mla-metric'
# bucket_id = 'mla-metric'
ip_addr_name = 'rsa-prod'

# These are parameters you setup for this deployment
region = 'us-central1'
zone = 'us-central1-f'


In [6]:
import os
os.environ['DEVSHELL_PROJECT_ID'] = project_id
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

!gcloud config set project {project_id} -q

Updated property [core/project].


In [7]:
!gcloud config set compute/zone {zone}
!gcloud config set compute/region {region}

Updated property [compute/zone].
Updated property [compute/region].


### Install Kubectl

In [8]:
!curl -LO https://storage.googleapis.com/kubernetes-release/release/`curl -s \
 https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.9M  100 41.9M    0     0   144M      0 --:--:-- --:--:-- --:--:--  144M


In [9]:
!chmod +x kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version --client

Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.6", GitCommit:"dff82dc0de47299ab66c83c626e08b245ab19037", GitTreeState:"clean", BuildDate:"2020-07-15T16:58:53Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}


##A.3 Enabling APIs

In [10]:
!gcloud services enable container.googleapis.com --async
!gcloud services enable containerregistry.googleapis.com --async
!gcloud services enable compute.googleapis.com --async
!gcloud services enable storage-api.googleapis.com --async
!gcloud services enable storage-component.googleapis.com --async                        
!gcloud services enable servicemanagement.googleapis.com --async
!gcloud services enable iam.googleapis.com --async
!gcloud services enable cloudbuild.googleapis.com --async

# B : Git Repository Pull

In [32]:
%cd ~
!rm -rf ~/RSA-Mono-Site
!git clone https://github.com/cfphpflex/RSA-Mono-Site.git
%cd ~/RSA-Mono-Site
!./setup.sh

/root
Cloning into 'RSA-Mono-Site'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 194 (delta 82), reused 161 (delta 52), pack-reused 0
Receiving objects: 100% (194/194), 1.26 MiB | 17.63 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/root/RSA-Mono-Site
Checking for required npm version...Completed.
Installing monolith dependencies...Completed.
Installing microservies dependencies...Completed.
Installing React app dependencies...Completed.
Building React app and placing into sub projects...Completed.

Script completed successfully!


In [33]:
!python main.py
! nano main.py


 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Traceback (most recent call last):
  File "main.py", line 29, in <module>
    app.run(host='127.0.0.1', port=8080, debug=True)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 1030, in run_simple
    s.bind(server_address)
OSError: [Errno 98] Address already in use
/bin/bash: nano: command not found


In [28]:
!ls -lrt

total 60
-rw-r--r-- 1 root root 3321 Jul 30 01:22 README.md
drwxr-xr-x 6 root root 4096 Jul 30 01:22 RSA-Core
-rwxr-xr-x 1 root root  768 Jul 30 01:22 main.py
-rwxr-xr-x 1 root root    9 Jul 30 01:22 index.html
-rwxr-xr-x 1 root root 2715 Jul 30 01:22 firestore.py
-rwxr-xr-x 1 root root 1336 Jul 30 01:22 deploy-monolith.sh
-rwxr-xr-x 1 root root 1049 Jul 30 01:22 app.yaml
-rwxr-xr-x 1 root root 2814 Jul 30 01:22 storage.py
-rwxr-xr-x 1 root root 1312 Jul 30 01:22 setup.sh
-rwxr-xr-x 1 root root  196 Jul 30 01:22 requirements.txt
drwxr-xr-x 3 root root 4096 Jul 30 01:23 microservices
drwxr-xr-x 2 root root 4096 Jul 30 01:23 logs
drwxr-xr-x 7 root root 4096 Jul 30 01:23 react-app
drwxr-xr-x 2 root root 4096 Jul 30 01:37 __pycache__
drwxr-xr-x 7 root root 4096 Jul 30 01:48 monolith


In [36]:
!gcloud container clusters create fancy-cluster --num-nodes 3

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
ERROR: (gcloud.container.clusters.create) ResponseError: code=409, message=Already exists: projects/rsa-mono-site5/zones/us-central1-f/clusters/fancy-cluster.


In [39]:
!gcloud config set container/use_application_default_credentials true

Updated property [container/use_application_default_credentials].


In [44]:
!gcloud compute instances list

NAME                                          ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
gke-fancy-cluster-default-pool-919ccec6-38xx  us-central1-f  n1-standard-1               10.128.0.7   34.67.206.206  RUNNING
gke-fancy-cluster-default-pool-919ccec6-kwwj  us-central1-f  n1-standard-1               10.128.0.6   34.67.199.234  RUNNING
gke-fancy-cluster-default-pool-919ccec6-qg7h  us-central1-f  n1-standard-1               10.128.0.5   35.184.32.75   RUNNING


Get Credentials

In [41]:
!gcloud config set container/use_application_default_credentials true
!gcloud container clusters get-credentials fancy-cluster
!kubectl cluster-info

Updated property [container/use_application_default_credentials].
Fetching cluster endpoint and auth data.
kubeconfig entry generated for fancy-cluster.
Kubernetes master is running at https://35.193.46.45
GLBCDefaultBackend is running at https://35.193.46.45/api/v1/namespaces/kube-system/services/default-http-backend:http/proxy
Heapster is running at https://35.193.46.45/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.193.46.45/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://35.193.46.45/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [43]:
!pwd
!python main.py
!./deploy-monolith.sh





Enabling Cloud Build APIs...Completed.
Building Monolith Container...Completed.
Deploying Monolith To GKE Cluster...Completed.

Please run the following command to find the IP address for the monolith service:  kubectl get service monolith

Deployment Complete


In [48]:
!gcloud compute instances list
!kubectl get service

NAME                                          ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
gke-fancy-cluster-default-pool-919ccec6-38xx  us-central1-f  n1-standard-1               10.128.0.7   34.67.206.206  RUNNING
gke-fancy-cluster-default-pool-919ccec6-kwwj  us-central1-f  n1-standard-1               10.128.0.6   34.67.199.234  RUNNING
gke-fancy-cluster-default-pool-919ccec6-qg7h  us-central1-f  n1-standard-1               10.128.0.5   35.184.32.75   RUNNING
NAME         TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
kubernetes   ClusterIP      10.19.240.1     <none>          443/TCP        46m
monolith     LoadBalancer   10.19.241.126   34.68.201.208   80:30302/TCP   10m


# C: Build Container Image

##Build Order Container Image and Deploy It

In [ ]:
%cd ~/monolith-to-microservices/microservices/src/orders
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0 .

In [ ]:
!kubectl create deployment orders --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0

In [ ]:
!kubectl expose deployment orders --type=LoadBalancer --port 80 --target-port 8081

##Build Product Container Image and Deploy It

In [ ]:
%cd ~/monolith-to-microservices/microservices/src/products
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0 .

In [ ]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

In [ ]:
!kubectl expose deployment products --type=LoadBalancer --port 80 --target-port 8082

## Build Frontend Container and Deploy It

In [47]:
!kubectl get service

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
kubernetes   ClusterIP      10.19.240.1     <none>          443/TCP        44m
monolith     LoadBalancer   10.19.241.126   34.68.201.208   80:30302/TCP   8m56s


### Copy the IP address of order and product LoadBalancer and paste it below

In [ ]:
ORDER_SERVICE_IP_ARR =!kubectl get -o jsonpath="{.status.loadBalancer.ingress[0].ip}" --namespace=default services orders
ORDER_SERVICE_IP = ORDER_SERVICE_IP_ARR[0]
print(ORDER_SERVICE_IP)
PROD_SERVICE_IP_ARR =!kubectl get -o jsonpath="{.status.loadBalancer.ingress[0].ip}" --namespace=default services products
PROD_SERVICE_IP = PROD_SERVICE_IP_ARR[0]
print(PROD_SERVICE_IP)

In [ ]:
%cd ~/monolith-to-microservices/react-app
!cp ./.env.monolith ./.env
!sed -i 's+/service/orders+http://{ORDER_SERVICE_IP}/api/orders+g' ./.env
!sed -i 's+/service/products+http://{PROD_SERVICE_IP}/api/products+g' ./.env
!cat ./.env

In [ ]:
!npm run build

In [ ]:
%cd ~/monolith-to-microservices/microservices/src/frontend
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0 .

In [ ]:
!kubectl create deployment frontend --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0

In [ ]:
!kubectl expose deployment frontend --type=LoadBalancer --port 80 --target-port 8080

## How can I Get the External-IP address?

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get pods

In [ ]:
!kubectl get service

In [ ]:
FRONT_SERVICE_IP_ARR=!kubectl get -o jsonpath="{.status.loadBalancer.ingress[0].ip}" --namespace=default services frontend
FRONT_SERVICE_IP = FRONT_SERVICE_IP_ARR[0]
'http://'+FRONT_SERVICE_IP

## Resilency Test

In [ ]:
!kubectl get pods

In [ ]:
!kubectl delete pod products-6fd4b78888-mqfwv


In [ ]:
!kubectl delete deployment products
#!kubectl delete

In [ ]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

In [ ]:
!curl {PROD_SERVICE_IP}/api/products

## Internal Name space

# C : Cleanup Project

###The code below will cause an error. It will prevent you from deleting the cluster unintentionally. <br>
###Execute the cluster delete when you are done to clean up.

In [ ]:
raise

In [ ]:
!gcloud container clusters list

In [ ]:
!gcloud container clusters delete fancy-cluster
